In [226]:
from abc import ABC, abstractmethod
from IPython.display import display, clear_output
from typing import List, Any, Optional
import numpy as np 
import pandas as pd 
from pathlib import Path
from lib.logger import log

class DisplayAPI(ABC):
    @abstractmethod
    def display(self):
        pass
    
class Print(DisplayAPI):
    def display(self, some_shit):
        print(some_shit.data)
    
class Display(DisplayAPI):
    def display(self, some_shit):
        display(some_shit.data)
    
class RawData(ABC):
    def __init__(self, file, api: Optional[DisplayAPI] = Display() ) -> None:
        self.file = file
        self.api = api
        
        ###
        self.get_data(file)
     
    def display(self):
        self.api.display(self)
        
    @abstractmethod
    def get_data(self, file):
        pass
    
    @property
    def text(self):
        return self.file.read_text()
    @property
    def lines(self):
        return self.text.splitlines()

    
class CSVFile(RawData):
    def get_data(self,  file, header, delimiter):
        header_line, skip = header
        header = self.lines[header_line].split(delimiter)
        data = np.genfromtxt((x.replace(',', '.').encode() for x in open(file)),skip_header=header_line+skip, delimiter=delimiter)
        
        df = pd.DataFrame(data[:,1:], index=data[:,0], columns=header[1:])
        df.index.name=header[0]

        self.data=df
    
    def get_sections(self) -> List[Any]:
        indexes = [ *[k for k,line in enumerate(self.lines) if line==''], -1]
        return [self.lines[i+1:k] for i,k in zip(indexes[0::2], indexes[1::2])]

class Avantes(CSVFile):
    def get_data(self, file, header=(5,3), delimiter=';'):
        return super().get_data(file, header, delimiter)
        
class Hitachi(CSVFile):
    def get_data(self, file, header=(30,1), delimiter='\t'):
        return super().get_data(file, header, delimiter)
       
absorbance_csv =  Path('../patka-pomiary/CPE45_PFOBPy_comocat.txt')
fluorescence_csv =  Path('../patka-pomiary/func39.txt')

rd = Avantes( file=fluorescence_csv, api=Display() ) # api=Display() is set by default 
rd2 = Hitachi( file=absorbance_csv, api=Display() )

rd.display()
rd2.display()

TypeError: get_data() takes 2 positional arguments but 4 were given